## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node 52ceb7f5-6164-4bf6-b28a-33899f775678

## Importation of libraries

In [2]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *

## Main script

In [3]:
u_old = 0
mu_predict_old = 0
cov_predict_old = 0
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
obstacle = 0

map = Map()

## map.obstacles = [[corner_1, ..., corner_p],
##                  [          ...          ],
#                   [corner_1, ..., corner_p]]

## map.target_lines = [[x1, y1]
#                      [x2, y2],
##                     [ ...  ],
#                      [x2, y2]]

## map.obstacles_lines = [[(x1, y1), (x2, y2)],
##                        [        ...       ],
#                         [(x1, y1), (x2, y2)]]

while True:
    map.show()
    if cv.waitKey(20) & 0xFF==ord('d'):
        break

input("Place the Thymio and the destination. Then, press Enter.")

while active == True:
    #print(map.target_lines)
    #print(map.obstacles_lines)
    #active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    #await node.wait_for_variables()
    #vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    #y = np.array([map.robot[0]], [map.robot[1]], [vel_meas[0]], [vel_meas[1]])
    #x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, view_blocked)
    #pose_est = x_est[0:2]
    #pose_est.append(map.robot[2])
    #error_est = cov_est[0:2]
    #error_est.append(0)

    # Global Navigation
    #create path
    # Local Navigation
    #u, obstacle, segment_index = collision_avoidance(path, pose_est, error_est, old_obstacle, segment_index) # Zacharie

    # Global Navigation
    if start == True:
        ex_path, corners = compute_visibility_matrix(map.robot[0:2], map.destination[0:2], map.obstacles)
        map.obstacles_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        start = False

    if not obstacle:
        u, segment_index, end = path_direction(map.robot, path, segment_index)
        #u = np.round(u/100)

    await node.set_variables(motors(int(u[0]), int(u[1])))

    #u_old = u
    #cov_predict_old = cov_est
    #mu_predict_old = x_est
    #obstacle_old = obstacle

    if cv.waitKey(20) & 0xFF==ord('p'):
        map.vision_stop()
        break

map.info()
cv.destroyAllWindows()


arrive fin segment
304.0
seg index: 1
nombre de seg 2
arrive fin segment
399.0
seg index: 2
nombre de seg 2
final destination


IndexError: index 3 is out of bounds for axis 0 with size 3

## Motors control test

In [ ]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    #vel_meas.append(velocity_with_odometry(node, 0)[0])
    await node.set_variables(motors(-100, 100))
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[-100:])
vel_meas_var = np.var(vel_meas[-100:])
print("Velocity measured:", vel_meas)
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-1010],
 '_productId': [8],
 'button.backward': [1],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [1],
 'button.right': [1],
 'buttons._raw': [11231, 9659, 13845, 14539, 15964],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-1010],
 '_imot': [-290, -335],
 '_vbat': [674, 681],
 'buttons._mean': [11713, 10030, 14926, 15068, 16548],
 'buttons._noise': [300, 263, 427, 415, 431],
 'motor.left.speed': [350],
 'motor.left.target': [384],
 'motor.right.speed': [390],
 'motor.right.target': [644],
 'prox.comm.rx': [0],
 'prox.comm.rx._intensities': [0, 0, 0, 0, 0, 0, 0],
 'prox.comm.rx._payloads': [0, 0, 0, 0, 0, 0, 0],
 'prox.comm.tx': [0],
 'prox.ground.ambiant': [0, 0],
 'prox.ground.delta': [0, 0],
 'prox.ground.reflected': [0, 0],
 'prox.horizontal': [0, 0, 0, 0, 0, 0, 0],
 '_integrator': [-213